In [ ]:
#NLP_Proj_데이콘 쇼핑몰 리뷰 평점 분류

#데이터 분석 & 예측

In [ ]:
import pandas as pd

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/commit/Dacon/nlp_review/train.csv')
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/commit/Dacon/nlp_review/test.csv')


In [ ]:
train.head()

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,4,튼튼하고 손목을 잘 받쳐주네요~,5


In [ ]:
test.head()

,id,reviews
0,0,채소가 약간 시들어 있어요
1,1,발톱 두껍고 단단한 분들 써도 소용없어요 이 테이프 물렁거리고 힘이없어서 들어 올리...
2,2,부들부들 좋네요 입어보고 시원하면 또 살게요
3,3,이런 1. 8 골드 주라니깐 파란개 오네 회사전화걸어도 받지도 않고 머하자는거임?
4,4,검수도 없이 보내구 불량 배송비 5000원 청구하네요 완전별로 별하나도 아까워요


In [ ]:
X=train.reviews
y=train.target

#텍스트 전처리

In [ ]:
# sklearn 패키지의 CountVectorizer를 불러옵니다
from sklearn.feature_extraction.text import CountVectorizer

sample_vectorizer = CountVectorizer()

이제, 어떠한 단어들을 사용할 지, 어떠한 단어가 중요도를 가지는지 학습하는 과정.
<br>CountVectorizer의 경우 단어의 출현 빈도를 기준으로 문장을 벡터(=숫자의 집합)으로 바꿔줍니다.

In [ ]:
sample_text1 = ["hello, my name is dacon and I am a data scientist!"]

sample_vectorizer.fit(sample_text1) #CountVectorizer 학습

CountVectorizer()

In [ ]:
print(sample_vectorizer.vocabulary_) #Vocabulary

{'hello': 4, 'my': 6, 'name': 7, 'is': 5, 'dacon': 2, 'and': 1, 'am': 0, 'data': 3, 'scientist': 8}


특수 문자가 의미가 적다고 판단하여 제외.

In [ ]:
sample_text2 = ["you are learning dacon data science"]

sample_vector = sample_vectorizer.transform(sample_text2)
print(sample_vector.toarray())

[[0 0 1 1 0 0 0 0 0]]


학습시킨 모델과 비교대조.

In [ ]:
# sklearn 패키지의 CountVectorizer import
from sklearn.feature_extraction.text import CountVectorizer

# countvectorizer 생성
vectorizer = CountVectorizer()

# countvectorizer 학습
vectorizer.fit(train['reviews'])

CountVectorizer()

#모델링
모델만들기 전에 모델이 유의미한지 테스트해야한다.
-> 따라서 train의 70%를 학습, 25%는 유의미한지 검증. 유의미하다면 실제로 제출할 모델을 만들 때, train셋 100%학습시키면된다.


In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(train, test_size=0.25, stratify = train.target) # 25프로로 설정

# 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화
train_data = train_data.reset_index().drop('index', axis=1)
val_data = val_data.reset_index().drop('index', axis=1)

In [ ]:
print('학습시킬 train 셋 : ', train_data.shape)
print('검증할 val 셋 : ', val_data.shape)

학습시킬 train 셋 :  (18750, 3)
검증할 val 셋 :  (6250, 3)


In [ ]:
# training 데이터에서 변수 추출
train_X = train_data.reviews
train_y = train_data.target

# validation 데이터에서 변수 추출
val_X = val_data.reviews
val_y = val_data.target

In [ ]:
# CountVectorizer 적용
train_X = vectorizer.transform(train_X)
val_X = vectorizer.transform(val_X)

In [ ]:
print(train_X[10])

# inverse_transform을 사용해 원문을 확인할 수 있다.
vectorizer.inverse_transform(train_X[10])

  (0, 11055)	1
  (0, 36686)	1
  (0, 60468)	1


[array(['깨끗하고', '상품', '좋습니다'], dtype='<U100')]

#Loigstic Regression

연속적인 값을 갖는 수치 형태가 아닌, 0/1 단위로 끊어지는 분류 문제
<br> category를 분류할 때 사용하는 LogisticRegression 활용

In [ ]:
from sklearn.linear_model import LogisticRegression #모델 불러오기

model = LogisticRegression(max_iter=500) #객체에 모델 할당
model.fit(train_X, train_y) #모델 학습

LogisticRegression(max_iter=500)

#모델 평가
평가지표(Metric)를 통해 확인한다.


In [ ]:
#Define Metirc
import numpy as np

def ACCURACY(true, pred):
    score = np.mean(true==pred)
    return score

##예측값 생성하여 실제 정답값과 비교

In [ ]:
# val_X를 이용해 예측값을 생성
y_pred = model.predict(val_X)

In [ ]:
# 모델의 예측과 실제 정답값을 비교
score = ACCURACY(val_y, y_pred)

print(f"모델의 정확도는 {score*100:.2f}% 입니다")

모델의 정확도는 61.22% 입니다
